# Overall US COVID-19 Vaccine administration data by juridiction, counties

Data represents all vaccine partners including jurisdictional partner clinics, retail pharmacies, long-term care facilities, dialysis centers, Federal Emergency Management Agency and Health Resources and Services Administration partner sites, and federal entity facilities. 

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_latimes as lat
import datetime as dt

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.options.display.max_colwidth = None

In [4]:
today = dt.datetime.today().strftime("%Y%m%d")

In [5]:
# https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh
# https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-Jurisdi/unsk-b7fc

In [6]:
src = pd.read_csv(
    # county
    # "https://data.cdc.gov/api/views/8xkx-amqh/rows.csv?accessType=DOWNLOAD",
    "https://data.cdc.gov/api/views/unsk-b7fc/rows.csv?accessType=DOWNLOAD",
    low_memory=False,
    parse_dates=["Date"],
    dtype={"FIPS": "str"},
)

In [7]:
src.columns = src.columns.str.lower()

In [8]:
src["date"] = pd.to_datetime(src["date"], format="%Y-%m-%d")
src["date"] = pd.to_datetime(src["date"].dt.strftime("%Y-%m-%d"))
src["year"] = src["date"].dt.year
src["quarter"] = src["date"].dt.quarter
src["day"] = src["date"].dt.day
src["month"] = src["date"].dt.month
src["weekday"] = src["date"].dt.weekday
src["monthname"] = src["date"].dt.month_name()
src["month_year_full"] = src["date"].apply(lambda x: x.strftime("%B-%Y"))
src["month_year_full"] = pd.to_datetime(src["month_year_full"])

In [9]:
df = src.copy()
df.head()

,date,mmwr_week,location,distributed,distributed_janssen,distributed_moderna,distributed_pfizer,distributed_unk_manuf,dist_per_100k,distributed_per_100k_12plus,distributed_per_100k_18plus,distributed_per_100k_65plus,administered,administered_12plus,administered_18plus,administered_65plus,administered_janssen,administered_moderna,administered_pfizer,administered_unk_manuf,administered_fed_ltc,administered_fed_ltc_residents,administered_fed_ltc_staff,administered_fed_ltc_unk,administered_fed_ltc_dose1,administered_fed_ltc_dose1_residents,administered_fed_ltc_dose1_staff,administered_fed_ltc_dose1_unk,admin_per_100k,admin_per_100k_12plus,admin_per_100k_18plus,admin_per_100k_65plus,recip_administered,administered_dose1_recip,administered_dose1_pop_pct,administered_dose1_recip_12plus,administered_dose1_recip_12pluspop_pct,administered_dose1_recip_18plus,administered_dose1_recip_18pluspop_pct,administered_dose1_recip_65plus,administered_dose1_recip_65pluspop_pct,series_complete_yes,series_complete_pop_pct,series_complete_12plus,series_complete_12pluspop_pct,series_complete_18plus,series_complete_18pluspop_pct,series_complete_65plus,series_complete_65pluspop_pct,series_complete_janssen,series_complete_moderna,series_complete_pfizer,series_complete_unk_manuf,series_complete_janssen_12plus,series_complete_moderna_12plus,series_complete_pfizer_12plus,series_complete_unk_manuf_12plus,series_complete_janssen_18plus,series_complete_moderna_18plus,series_complete_pfizer_18plus,series_complete_unk_manuf_18plus,series_complete_janssen_65plus,series_complete_moderna_65plus,series_complete_pfizer_65plus,series_complete_unk_manuf_65plus,series_complete_fedltc,series_complete_fedltc_residents,series_complete_fedltc_staff,series_complete_fedltc_unknown,year,quarter,day,month,weekday,monthname,month_year_full
0,2021-06-21,25,MA,9628260,444700,3851480,5332080,0,139692,160024,173805,823398,8680160,8657236,8187450,2058010,284229,3496257,4899479,195,320932,143582,132301,45049,180541,77201,71291,32049,125936,143886,147796,175999,8673655,4795585,69.6,4781257,79.5,4510535,81.4,1121411,95.9,4135231,60.0,4126778,68.6,3926056,70.9,1000321,85.5,284896,1603187,2247089,59,284891,1603179,2238649,59,283815,1598259,2043934,48,63646,456263,480393,19,138695,65517,60043,13135,2021,2,21,6,0,June,2021-06-01
1,2021-06-21,25,KS,3057805,173500,1340880,1543425,0,104960,124722,138171,643089,2442071,2441960,2358630,776885,80079,1079895,1281420,677,90005,45974,28032,15999,50647,24656,16796,9195,83825,99603,106578,163387,2547334,1413782,48.5,1413673,57.7,1358467,61.4,434300,91.3,1194155,41.0,1194135,48.7,1159452,52.4,373312,78.5,79926,506191,607792,246,79920,506184,607785,246,79861,506025,573325,241,14144,181750,177284,134,38922,20970,11125,6827,2021,2,21,6,0,June,2021-06-01
2,2021-06-21,25,NJ,11412635,606300,4683140,6123195,0,128489,149804,164362,773457,9893949,9893330,9400022,2440457,377120,4101838,5414516,475,263669,128442,108771,26456,153869,73609,62754,17506,111391,129861,135377,165395,10175017,5670194,63.8,5669567,74.4,5379559,77.5,1351397,91.6,4849487,54.6,4849379,63.7,4642222,66.9,1215778,82.4,380712,1909782,2558944,49,380665,1909762,2558903,49,380526,1909436,2352220,40,79202,575907,560648,21,110388,55127,45953,9308,2021,2,21,6,0,June,2021-06-01
3,2021-06-21,25,SD,952065,59400,428760,463905,0,107619,128988,142619,626891,805837,805814,778393,260304,24770,356554,424500,13,12569,7709,2101,2759,7346,4583,1384,1379,91090,109174,116603,171398,810080,440541,49.8,440517,59.7,423249,63.4,138116,90.9,393528,44.5,393518,53.3,383421,57.4,129627,85.4,24978,170780,197756,14,24977,170778,197749,14,24966,170737,187704,14,3576,66938,59106,7,5270,3121,729,1420,2021,2,21,6,0,June,2021-06-01
4,2021-06-21,25,BP2,200890,10600,83820,106470,0,0,0,0,0,192592,192591,192591,6311,8085,79267,105240,0,0,0,0,0,0,0,0,0,0,0,0,0,192592,104830,0.0,104829,0.0,104829,0.0,3309,0.0,94810,0.0,94810,0.0,94810,0.0,3067,0.0,8042,37173,49595,0,8042,37173,49595,0,8042,37173,49595,0,103,1498,1466,0,0,0,0,0,2021,2,

In [10]:
week_then = [
    "2021-05-27",
    "2021-05-28",
    "2021-05-29",
    "2021-05-30",
    "2021-05-31",
    "2021-06-01",
    "2021-06-02",
]
week_now = [
    "2021-06-03",
    "2021-06-04",
    "2021-06-05",
    "2021-06-06",
    "2021-06-07",
    "2021-06-08",
    "2021-06-09",
]

### Group by state and sum the administered doses

In [11]:
states = df.groupby(["date", "location"]).agg({"administered": sum}).reset_index()

### Isolate our specific weeks

In [12]:
state_week_then = states[(states["date"].isin(week_then))].sort_values(
    ["location", "date"]
)

In [13]:
state_week_now = states[(states["date"].isin(week_now))].sort_values(
    ["location", "date"]
)

### Calculate the change for each week

In [14]:
state_week_then["change"] = state_week_then.groupby("location")["administered"].diff(
    periods=6
)

In [15]:
state_week_now["change"] = state_week_now.groupby("location")["administered"].diff(
    periods=6
)

### Combine the two dataframes

In [16]:
state_changes_weeks = pd.concat(
    [
        state_week_now[state_week_now["date"] == "2021-06-09"],
        state_week_then[state_week_then["date"] == "2021-06-02"],
    ]
).sort_values(["location", "date"], ascending=True)

### Calculate the pct change from week to week

In [17]:
state_changes_weeks["change"] = (
    state_changes_weeks.groupby(["location"])["administered"].pct_change(1) * 100
).round(2)

### Export the result

In [18]:
state_changes_weeks[state_changes_weeks["date"] == "2021-06-09"].sort_values(
    "change", ascending=False
).to_csv(
    "output/vaccine-community/state_vax_mean_admin_change_05-27to06-02__06-03to06-06.csv",
    index=False,
)